<h1>geopandas-working-with-attributes </h1>

- The goal of this small project is that we download the population data and mark the places where its population is gerater than 1 million onto the map and save the resultant data as GeoJSON file.


- The data source is <a href='https://www.qgistutorials.com/en/docs/3/working_with_attributes.html'> Working with Attributes (QGIS3) </a> 
- Here, we are going to use GeoPandas, not QGIS3
- Geographic Information System (GIS) has two parts: features and attributes.
    - Attributes are structured data about each feature
    
    
    
- Data
    - Populated Places <a href='http://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-populated-places/'> here </a> : Please download this "ne_10m_populated_places_simple.zip"
        

In [1]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from urllib import request
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

%config InlineBackend.figure_format='svg'
plt.rcParams['figure.dpi']=120

pd.options.display.float_format='{:,.2f}'.format
pd.set_option('display.max_colwidth', None)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
url = "https://www.qgistutorials.com/downloads/ne_10m_populated_places_simple.zip"
fname = 'population_places.zip'
request.urlretrieve(url, fname)

('population_places.zip', <http.client.HTTPMessage at 0x27253324e20>)

In [3]:
from zipfile import ZipFile

file_name = "population_places.zip"

with ZipFile(file_name, 'r') as zip:
    # zip.printdir()
    zip.extractall()


In [4]:
import os

os.listdir('.')

['.ipynb_checkpoints',
 'from_github',
 'geopandas_working_with_attributes.ipynb',
 'ne_10m_populated_places_simple.cpg',
 'ne_10m_populated_places_simple.dbf',
 'ne_10m_populated_places_simple.prj',
 'ne_10m_populated_places_simple.README.html',
 'ne_10m_populated_places_simple.shp',
 'ne_10m_populated_places_simple.shx',
 'ne_10m_populated_places_simple.VERSION.txt',
 'populated_capitals.geojson',
 'population_places.zip']

**Create geopandas dataframe**

In [5]:
fname = 'ne_10m_populated_places_simple.shp'
gdf = gpd.read_file(fname)

In [6]:
gdf.head(3)

,scalerank,natscale,labelrank,featurecla,name,namepar,namealt,diffascii,nameascii,adm0cap,...,rank_max,rank_min,geonameid,meganame,ls_name,ls_match,checkme,min_zoom,ne_id,geometry
0,10,1,8,Admin-1 capital,Colonia del Sacramento,None,None,0,Colonia del Sacramento,0.00,...,7,7,"3,443,013.00",None,None,0,0,9.00,1159112629,POINT (-57.84000 -34.48000)
1,10,1,8,Admin-1 capital,Trinidad,None,None,0,Trinidad,0.00,...,7,7,"3,439,749.00",None,None,0,0,9.00,1159112647,POINT (-56.90100 -33.54400)
2,10,1,8,Admin-1 capital,Fray Bentos,None,None,0,Fray Bentos,0.00,...,7,7,"3,442,568.00",None,None,0,0,9.00,1159112663,POINT (-58.30400 -33.13900)


In [7]:
gdf['adm0cap'].unique() # when the country has country capita data then it is marked as 1 otherwise it is 0

array([0., 1.])

In [8]:
gdf.columns

Index(['scalerank', 'natscale', 'labelrank', 'featurecla', 'name', 'namepar',
       'namealt', 'diffascii', 'nameascii', 'adm0cap', 'capalt', 'capin',
       'worldcity', 'megacity', 'sov0name', 'sov_a3', 'adm0name', 'adm0_a3',
       'adm1name', 'iso_a2', 'note', 'latitude', 'longitude', 'changed',
       'namediff', 'diffnote', 'pop_max', 'pop_min', 'pop_other', 'rank_max',
       'rank_min', 'geonameid', 'meganame', 'ls_name', 'ls_match', 'checkme',
       'min_zoom', 'ne_id', 'geometry'],
      dtype='object')

In [9]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

**explore() method allows your map to be interactive map**

- The basemap can be found in a website ("leaflet Provide Demo")

- Or we can get the basemaps from Hatarilab ("https://hatarilabs.com/ih-en/how-to-add-a-google-map-in-qgis-3-tutorial"

In [10]:
gdf.head(100).explore(tiles='http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}',
                      attr='Google Satellite', marker_kwds={'radius':10},
                     style_kwds={'fillColor':'red'}, popup=['name','adm0cap','pop_max'])

**Extracting data which has country capita with its population being greater than 1 million**

In [11]:
filt=(gdf['pop_max']>1000000) & (gdf['adm0cap'] ==1)

In [12]:
gdf_filtered = gdf[filt]

In [13]:
gdf_filtered.shape

(103, 39)

In [14]:
gdf_filtered.explore(tiles='http://mt0.google.com/vt/lyrs=p&hl=en&x={x}&y={y}&z={z}',
                      attr='Google Terrain', marker_kwds={'radius':10},
                     style_kwds={'fillColor':'red'},popup=['name','pop_max'])

In [15]:
gdf_filtered.explore(tiles='http://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={z}',
                      attr='Google Satellite', marker_kwds={'radius':10},
                     style_kwds={'fillColor':'red'},popup=['name','pop_max'])

**Export data as `GeoJSON`**

In [16]:
gdf_filtered = gdf_filtered[['name', 'pop_max','geometry']]

gdf_filtered_wrong = gdf_filtered[['name', 'pop_max']]

In [17]:
type(gdf_filtered), type(gdf_filtered_wrong)

(geopandas.geodataframe.GeoDataFrame, pandas.core.frame.DataFrame)

In [18]:
gdf_filtered.to_file('populated_capitals.geojson', driver='GeoJSON')

In [19]:
gpd.read_file('populated_capitals.geojson').explore()

In [20]:
import fiona

In [21]:
fiona.supported_drivers

{'ARCGEN': 'r',
 'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'r',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'rw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'rw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'GPSTrackMaker': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'raw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}